In [0]:
%pip install pytest

In [0]:
import pytest
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col

table_name = "nintendodatabricks_workspace.silver.consoles_silver"
expected_columns = ["codigo", "nome", "preco", "desconto",
                    "numero_parcelas", "valor_prestacao", "origem", "link",
                    "memoria", "oled", "extract", "recomendacao"]

expected_schema_types = {
    "codigo": "string",
    "nome": "string",
    "preco": "double",
    "desconto": "double",
    "numero_parcelas": "bigint",
    "valor_prestacao": "double",
    "origem": "string",
    "link": "string",
    "memoria": "string",
    "oled": "string",
    "extract": "timestamp",
    "recomendacao": "string"
}

def check_table_exists(table):
    try:
        spark.table(table)
        return True
    except AnalysisException:
        return False

def check_schema(table, expected_cols):
    df = spark.table(table)
    actual_cols = df.columns
    return set(actual_cols) == set(expected_cols)

def check_not_empty(table):
    df = spark.table(table)
    return not df.rdd.isEmpty()

def check_column_types(table, expected_types):
    df = spark.table(table)
    actual_types = {field.name: field.dataType.simpleString() for field in df.schema.fields}
    for col, expected_type in expected_types.items():
        if col not in actual_types or actual_types[col] != expected_type:
            return False
    return True

@pytest.mark.parametrize("table", [table_name])
def test_table_exists(table):
    assert check_table_exists(table), f"Table {table} should exist"

@pytest.mark.parametrize("table,expected_cols", [(table_name, expected_columns)])
def test_schema(table, expected_cols):
    assert check_schema(table, expected_cols), f"Schema of table {table} does not match expected"

@pytest.mark.parametrize("table", [table_name])
def test_not_empty(table):
    assert check_not_empty(table), f"Table {table} is empty"

@pytest.mark.parametrize("table,expected_types", [(table_name, expected_schema_types)])
def test_column_types(table, expected_types):
    assert check_column_types(table, expected_types), f"Column types of table {table} do not match expected"

In [0]:
print("Table exists:", check_table_exists(table_name))
print("Schema ok:", check_schema(table_name, expected_columns))
print("Type ok:", check_column_types(table_name, expected_schema_types))
print("Not empty:", check_not_empty(table_name))


In [0]:
try:
    test_table_exists(table_name)
    test_schema(table_name, expected_columns)
    test_schema(table_name, expected_schema_types)
    test_not_empty(table_name)
    print("✅ Todos os testes passaram")
except AssertionError as e:
    print(f"❌ Falhou: {e}")
